## Genome-Wide SNP Matching Setup

**Created**: 6 June 2022

## Environment

In [1]:
library(tidyverse)
library(data.table)
library(rtracklayer)
library(GenomicRanges)

setwd("~/eQTL_pQTL_Characterization/")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGeneri

## Load Data

In [2]:
cis.eqtl <- read.table("~/gains_team282/eqtl/cisresults/eigenMT/ciseqtl_eigenMT_corrected.txt") %>%
    dplyr::filter(Sig)

In [3]:
head(cis.eqtl)

,snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,BF,TESTS,BF.FDR,Sig,threshold
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<dbl>
22,rs3131972,ENSG00000237491,5.643405,2.400875e-08,0.07174216,0.012712570,1,817341,778747,7.058573e-06,294,1.993414e-05,TRUE,9.170798e-05
23,rs3131972,ENSG00000230092,4.756587,2.329109e-06,0.06239499,0.013117600,1,817341,817712,6.917454e-04,297,1.593483e-03,TRUE,9.078164e-05
25,rs3131972,ENSG00000225880,6.292961,5.949542e-10,0.10016770,0.015917420,1,817341,827522,1.767014e-07,297,5.668498e-07,TRUE,9.078164e-05
28,rs2272757,ENSG00000188976,-7.026045,4.904859e-12,-0.05107767,0.007269761,1,946247,959309,1.545031e-09,315,5.762103e-09,TRUE,8.559411e-05
29,rs13303327,ENSG00000187961,-7.175012,1.845538e-12,-0.13519920,0.018843060,1,960326,960584,5.813445e-10,315,2.228468e-09,TRUE,8.559411e-05
30,rs13303056,ENSG00000187583,-9.081031,1.922802e-18,-0.29633770,0.032632610,1,953778,966482,6.056826e-16,315,3.262796e-15,TRUE,8.559411e-05


In [4]:
c.cis.eqtl <- readRDS("~/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds")

In [5]:
head(c.cis.eqtl)

,SNP,Gene,eQTL_beta,eQTL_SE,pvalue,Number
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,rs10753794,ENSG00000000457,0.0498587312408011,0.00708515851579373,5.19259471106013e-12,1
2,rs10919255,ENSG00000000460,-0.0726264894633498,0.0136853304567293,1.72815915933528e-07,1
3,rs77006036,ENSG00000000460,0.166236949102582,0.0334472538699365,8.47290238610159e-07,2
4,rs12406047,ENSG00000000971,0.17135374034767,0.0379807691167313,7.51523462853891e-06,1
5,rs6696136,ENSG00000001460,0.45884434698694,0.0871081626842823,1.94887370606405e-07,1
6,rs6676449,ENSG00000001460,-0.371531778822554,0.0880672792106829,2.8471651589813e-05,2


In [6]:
ld.tags <- read.table("~/gains_team282/nikhil/colocalization/cis_eqtl/fine_mapping/LD/lead_snps.80r2.tags.tsv", header=TRUE) %>%
    dplyr::mutate(NTAG = NTAG + 1)

In [7]:
head(ld.tags)

,SNP,CHR,BP,NTAG,LEFT,RIGHT,KBSPAN,TAGS
,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
1,rs3131972,1,817341,1,817341,817341,0.001,NONE
2,rs2272757,1,946247,1,946247,946247,0.001,NONE
3,rs13303056,1,953778,34,941119,966227,25.109,rs4372192|rs6605067|rs2839|rs3748592|rs2340582|rs4246503|rs4970377|rs4970452|rs4970376|rs4970375|rs3748595|rs3828047|rs3748596|rs3748597|rs13302945|rs3828049|rs13303206|rs13303051|rs13302957|rs13303227|rs4970371|rs6605069|rs4970445|rs28393498|rs4970441|rs13303229|rs6696971|rs41285808|rs4970435|rs4970434|rs9697711|rs13303351|rs13302996
4,rs13303327,1,960326,5,951408,966179,14.772,rs10465242|rs13303010|rs3935066|rs13303160
5,rs13303033,1,979560,4,979560,983193,3.634,rs6694632|rs13303118|rs2341354
6,rs3128117,1,1009184,1,1009184,1009184,0.001,NONE


In [8]:
c.ld.tags <- read.table("~/gains_team282/nikhil/colocalization/cis_eqtl/fine_mapping/LD/conditional_snps.80r2.tags.tsv", header=TRUE) %>%
    dplyr::mutate(NTAG = NTAG + 1)

In [9]:
head(c.ld.tags)

,SNP,CHR,BP,NTAG,LEFT,RIGHT,KBSPAN,TAGS
,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
1,rs3131972,1,817341,1,817341,817341,0.001,NONE
2,rs28731045,1,901149,8,898547,903175,4.629,rs28593608|rs28385272|rs4422949|rs28570054|rs72890788|rs4970384|rs4970383
3,rs142336952,1,918014,1,918014,918014,0.001,NONE
4,rs2272757,1,946247,1,946247,946247,0.001,NONE
5,rs4970377,1,950296,34,941119,966227,25.109,rs4372192|rs6605067|rs2839|rs3748592|rs2340582|rs4246503|rs4970452|rs4970376|rs4970375|rs3748595|rs3828047|rs3748596|rs3748597|rs13303056|rs13302945|rs3828049|rs13303206|rs13303051|rs13302957|rs13303227|rs4970371|rs6605069|rs4970445|rs28393498|rs4970441|rs13303229|rs6696971|rs41285808|rs4970435|rs4970434|rs9697711|rs13303351|rs13302996
6,rs13303056,1,953778,34,941119,966227,25.109,rs4372192|rs6605067|rs2839|rs3748592|rs2340582|rs4246503|rs4970377|rs4970452|rs4970376|rs4970375|rs3748595|rs3828047|rs3748596|rs3748597|rs13302945|rs3828049|rs13303206|rs13303051|rs13302957|rs13303227|rs4970371|rs6605069|rs4970445|rs28393498|rs4970441|rs13303229|rs6696971|rs41285808|rs4970435|rs4970434|rs9697711|rs13303351|rs13302996


## SNPsnap

Use the [SNPsnap Web Server](https://data.broadinstitute.org/mpg/snpsnap/) to identify matching SNPs using the European 1000 Genomes population. Sample 10,000 SNPs for each SNP.

In [10]:
all.snps <- union(cis.eqtl$snps, c.cis.eqtl$SNP)
all.snps <- gsub("_.*$", "", gsub("\\.", ":", all.snps))

In [11]:
length(all.snps)

[1] 15825

In [12]:
write.table(all.snps, "~/gains_team282/epigenetics/enrichment/snp_snap/all_cis_eqtl_snps.txt", quote=F, row.names=F, col.names=F)